In [1]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [37]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from coset_bounds import *
from bound_utils import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:

device = t.device("cuda" if t.cuda.is_available() else "cpu")
# MODEL_DIR = '2024-08-16_22-53-29_SL2_5_MLP3_256'
MODEL_DIR = '2024-08-14_21-24-30_gapS_5_'
# MODEL_DIR = '2024-09-18_23-59-00_gapS5_MLP2_128_wd2e-4_ubias'
# MODEL_DIR = '2024-09-19_02-28-43_gapS5_MLP2_128_wd1e-4_ubias'
# MODEL_DIR = '2024-09-18_05-50-44_gapS5_MLP2_128_wd1e-5_ubias'
# MODEL_DIR = '2024-09-16_05-00-33_gapS5_MLP2_128_adamw_wd1'
# MODEL_DIR = '2024-09-13_03-46-47_gapS5_256_wd1e-3'
# MODEL_DIR = '2024-09-13_21-56-07_gapS5_MLP2_256_adamw_wd1'
# MODEL_DIR = '2024-08-27_16-32-46_Dic30_MLP3_128'
# MODEL_DIR = '2024-08-27_23-16-05_F11_MLP3_256'
# MODEL_DIR = '2024-09-02_18-17-20_smallgrp_54_8_MLP3_256_0_6'
# MODEL_DIR = '2024-09-05_15-22-58_smallgrp_54_8_MLP3_1024_0_6'
# MODEL_DIR = '2024-09-05_20-47-44_smallgrp_54_8_MLP3_1024_0_6_wd1e-6'
# MODEL_DIR = '2024-09-02_18-51-29_smallgrp_96_71_MLP3_256_0_6'
# MODEL_DIR = '2024-09-03_17-28-20_smallgrp_192_1025_MLP3_512_0_6'
# MODEL_DIR = '2024-09-03_21-09-07_Z_59_128'
disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir)
models = models[-1]  # get last checkpoint
data = GroupData(params)
group = data.groups[0]

Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: gapS(5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


/home/wilson/Finite-groups/src/model_utils.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location=device))


In [4]:
loss_dict = test_loss(models.to(device), data)

In [5]:
# instance = loss_dict['G0_acc'].argmax().item()
instance = loss_dict['G0_loss'].argmin().item()
# instance = (loss_dict['G0_acc'] > 0.999).nonzero()[11]
# instance = 2
print(loss_dict[f'G0_loss'][instance], loss_dict[f'G0_acc'][instance], instance)
model = models[instance].to(device)

tensor(0.0296) tensor(1.) 71


In [6]:
ln, rn, un = model.get_neurons(True)

In [32]:
left_neuron_subgroups, right_neuron_subgroups, conjugators, subgroups, left_cosets, right_cosets = get_neuron_subgroups(model, group, thresh=0.01)

Computing subgroups from gap_repr


Computing elements: 100%|██████████| 17/17 [00:00<00:00, 191.57it/s]


In [33]:
ideal_model = coset_ideal_model(model, group, left_neuron_subgroups, right_neuron_subgroups, conjugators, subgroups, left_cosets, right_cosets)

ln 1-r2 1.4677649232908152e-05
rn 1-r2 1.635864464333281e-05
un 1-r2 0.006147076841443777


In [40]:
bound, dists, margins = coset_bound(model, group, left_neuron_subgroups, right_neuron_subgroups, conjugators, subgroups, left_cosets, right_cosets)

ln 1-r2 1.4677649232908152e-05
rn 1-r2 1.635864464333281e-05
un 1-r2 0.006147076841443777
tensor([5.4119e-01, 5.4119e-01, 5.4119e-01, 1.4048e-04, 5.4119e-01, 5.4119e-01,
        1.0643e+00, 0.0000e+00, 4.4730e-01, 4.4730e-01, 4.4730e-01, 4.4730e-01,
        9.3886e-02, 9.3886e-02, 5.4119e-01, 8.9652e-02, 5.4119e-01, 5.4119e-01,
        9.3886e-02, 9.3886e-02, 8.9630e-02, 5.4119e-01, 5.4119e-01, 8.9652e-02,
        3.4891e-01, 0.0000e+00, 3.4891e-01, 3.4891e-01, 3.4877e-01, 3.4877e-01,
        2.5701e-01, 2.5701e-01, 2.5701e-01, 1.4049e-04, 2.5687e-01, 1.1454e+00,
        3.4891e-01, 8.7859e-02, 3.4891e-01, 9.2045e-02, 3.4877e-01, 9.1905e-02,
        8.7859e-02, 3.4891e-01, 3.4891e-01, 9.2045e-02, 9.1281e-02, 9.2045e-02,
        5.4933e-01, 5.4933e-01, 5.4933e-01, 1.4179e-04, 4.3370e-01, 1.1560e+00,
        1.1562e+00, 0.0000e+00, 2.3954e-01, 6.7609e-01, 6.7609e-01, 6.7609e-01,
        2.5661e-01, 6.4454e-01, 7.9195e-01, 4.9633e-01, 4.5395e-01, 8.9918e-01,
        6.9705e-01, 6.4454e-01

In [50]:
trace1 = go.Histogram(x=list(dists)*120, opacity=0.5, name='logit dist')
trace2 = go.Histogram(x=list(margins)*120, opacity=0.5, name='margin')
layout = go.Layout(
    # yaxis_type="log",
    width=800,
    height=500,
    title=dict(
        text='Margin vs logit distance bounds via, V_coset , S5',
        font=dict(size=24)  # Increase title font size
    ),
    xaxis=dict(
        title='Value',
        titlefont=dict(size=18),  # Increase x-axis title font size
        tickfont=dict(size=14),  # Increase x-axis tick label font size
    ),
    yaxis=dict(
        title='Count',
        titlefont=dict(size=18),  # Increase y-axis title font size
        tickfont=dict(size=14),  # Increase y-axis tick label font size
        #range=[0, 2500],
    ),
    legend=dict(font=dict(size=16)),
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [45]:
dists

tensor([5.0119, 5.3804, 5.4084, 5.0239, 5.0115, 5.3830, 5.4225, 5.0308, 5.0078,
        5.3657, 5.3974, 5.1496, 5.0922, 5.3891, 5.3914, 4.9946, 5.0201, 5.4578,
        5.3849, 5.0227, 5.0593, 5.3695, 5.4246, 5.0560, 5.4017, 5.0280, 5.0405,
        5.3658, 5.4195, 5.0878, 5.0470, 5.3679, 5.4228, 5.0531, 5.0281, 5.3902,
        5.4116, 5.0275, 5.0934, 5.4355, 5.4183, 5.0555, 5.0151, 5.3927, 5.3744,
        5.0347, 5.0207, 5.4327, 5.0474, 5.3860, 5.4035, 5.0286, 5.0402, 5.4244,
        5.4241, 5.0010, 5.0379, 5.3627, 5.4398, 5.0268, 5.1048, 5.4164, 5.4182,
        5.0592, 5.0688, 5.4348, 5.4186, 5.0519, 5.0262, 5.4086, 5.3872, 5.0226,
        5.4238, 5.0363, 5.0764, 5.3881, 5.4222, 5.0468, 5.0364, 5.3816, 5.4311,
        5.0323, 5.0917, 5.4541, 5.4801, 5.0669, 5.1027, 5.4189, 5.4098, 5.0874,
        5.0756, 5.4183, 5.3895, 5.0415, 5.0290, 5.4058, 5.0105, 5.3960, 5.3779,
        5.0281, 5.0195, 5.3964, 5.3954, 5.0377, 4.9964, 5.4122, 5.3737, 5.0024,
        5.0423, 5.3933, 5.3784, 5.0420, 

In [47]:
margins

tensor([5.4119e-01, 5.4119e-01, 5.4119e-01, 1.4048e-04, 5.4119e-01, 5.4119e-01,
        1.0643e+00, 0.0000e+00, 4.4730e-01, 4.4730e-01, 4.4730e-01, 4.4730e-01,
        9.3886e-02, 9.3886e-02, 5.4119e-01, 8.9652e-02, 5.4119e-01, 5.4119e-01,
        9.3886e-02, 9.3886e-02, 8.9630e-02, 5.4119e-01, 5.4119e-01, 8.9652e-02,
        3.4891e-01, 0.0000e+00, 3.4891e-01, 3.4891e-01, 3.4877e-01, 3.4877e-01,
        2.5701e-01, 2.5701e-01, 2.5701e-01, 1.4049e-04, 2.5687e-01, 1.1454e+00,
        3.4891e-01, 8.7859e-02, 3.4891e-01, 9.2045e-02, 3.4877e-01, 9.1905e-02,
        8.7859e-02, 3.4891e-01, 3.4891e-01, 9.2045e-02, 9.1281e-02, 9.2045e-02,
        5.4933e-01, 5.4933e-01, 5.4933e-01, 1.4179e-04, 4.3370e-01, 1.1560e+00,
        1.1562e+00, 0.0000e+00, 2.3954e-01, 6.7609e-01, 6.7609e-01, 6.7609e-01,
        2.5661e-01, 6.4454e-01, 7.9195e-01, 4.9633e-01, 4.5395e-01, 8.9918e-01,
        6.9705e-01, 6.4454e-01, 2.6037e-01, 7.9195e-01, 6.4454e-01, 4.5239e-01,
        7.9195e-01, 3.4599e-01, 3.3192e-

In [112]:
test_loss(model, data), test_loss(ideal_model, data)

({'G0_loss': tensor([0.0296]), 'G0_acc': tensor([1.])},
 {'G0_loss': tensor([0.1095]), 'G0_acc': tensor([1.])})

In [104]:
model.unembedding.shape, ideal_model.unembedding.shape

(torch.Size([1, 128, 120]), torch.Size([1, 128, 120]))

In [108]:
ideal_model.unembed_bias

Parameter containing:
tensor([], requires_grad=True)

In [30]:
model_dist_xy(model, ideal_model, 'inf', group=group, collapse_xy=True)

tensor([4.8886, 5.2738, 5.2409, 4.9091, 4.9055, 5.3158, 5.3145, 4.9640, 4.8884,
        5.3628, 5.2575, 4.9018, 4.8703, 5.2906, 5.2713, 4.8698, 4.9429, 5.1850,
        5.3097, 4.9010, 4.8734, 5.2393, 5.2609, 4.9169, 5.2342, 4.9089, 4.8324,
        5.2542, 5.2519, 4.8959, 4.8943, 5.2997, 5.2112, 4.9374, 4.8838, 5.3019,
        5.3342, 4.9130, 4.8653, 5.2545, 5.2261, 4.9107, 4.9085, 5.3024, 5.2548,
        4.9071, 4.8810, 5.2825, 4.9417, 5.2681, 5.2725, 4.9147, 4.8945, 5.3356,
        5.2945, 4.9694, 4.9122, 5.3376, 5.2240, 4.9045, 4.8769, 5.2788, 5.2135,
        4.9153, 4.8997, 5.2722, 5.3129, 4.9373, 4.8991, 5.2808, 5.2621, 4.8906,
        5.1980, 4.9282, 4.8505, 5.2490, 5.2735, 4.8508, 4.9312, 5.2680, 5.2321,
        4.9357, 4.8474, 5.1963, 5.2660, 4.9585, 4.8502, 5.2910, 5.2629, 4.9153,
        4.8844, 5.2336, 5.2788, 4.8957, 4.8715, 5.3105, 4.8594, 5.3283, 5.2772,
        4.9026, 4.8955, 5.2853, 5.2549, 4.9430, 4.8768, 5.3225, 5.2945, 4.8759,
        4.9290, 5.2820, 5.2498, 4.8931, 